In [18]:
import tensorflow as tf
import dataProcessing
import utils
import data
from models.refModel import RefConvNet

print('Tensorflow version: {}'.format(tf.__version__))

Tensorflow version: 2.0.0


In [13]:
seed = 1
tf.random.set_seed(seed)

In [14]:
 x_train, y_train, x_test, y_test = utils.getCifar10Dataset()
 x_train, y_train, x_test, y_test = dataProcessing.preprocessData(x_train, y_train, x_test, y_test,batch_size=128)
IMG_HEIGHT = x_test.shape[1]
IMG_WIDTH = x_test.shape[2]
CHANNELS = x_test.shape[3]

In [20]:
n_draw_images = 6
images = x_train[:n_draw_images]
labels = y_train[:n_draw_images]
utils.drawImages(images, labels)

IndexError: tuple index out of range

In [16]:
batch_size = 128
x_train, y_train = dataProcessing.createBatches(x_train, y_train, batch_size)

In [19]:
model = RefConvNet(32, input_shape=(IMG_HEIGHT, IMG_WIDTH, CHANNELS))